In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
%run -i ../../core.py

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.round(np.random.uniform(min(data), max(data), len(data)),2)
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.tensor(np.median(dummy_data))
# print(int(len(dummy_data)/2))
dummy_lower_to_median = torch.tensor(np.sort(dummy_data)[int(len(dummy_data)/2)-1])
dummy_upper_to_median = torch.tensor(np.sort(dummy_data)[int(len(dummy_data)/2)])


In [5]:
# print("dummy data:", dummy_data)
print("sort: ", np.sort(dummy_data)[149])
print("sort: ", np.sort(dummy_data)[150])
print("median: ", dummy_theory_output)
print("lower: ", dummy_lower_to_median)
print("upper: ", dummy_upper_to_median)

sort:  53.62
sort:  53.63
median:  tensor(53.6250, dtype=torch.float64)
lower:  tensor(53.6200, dtype=torch.float64)
upper:  tensor(53.6300, dtype=torch.float64)


In [6]:
print("dummy output: ", dummy_theory_output)
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.lower = nn.Parameter(data = dummy_lower_to_median, requires_grad = False)
        self.upper = nn.Parameter(data = dummy_upper_to_median, requires_grad = False)
    def forward(self,X):
        # some expression of tolerance to error in the inference
        count_less = torch.sum((X < self.w).double())
        count_equal = torch.sum((X == self.w).double())
        len = X.size()[1]
        half_len = torch.div(len, 2)

        # doesnt support mod if not in conditional
       
        # For count_equal > 0
        if(len%2==0):
            is_odd = 0
        else:
            is_odd = 1
        less_cons = count_less<half_len+is_odd
        more_cons = count_less+count_equal>half_len

        # For count_equal == 0
        lower_exist = torch.sum((X == self.lower).double())>0
        lower_cons = torch.sum((X>self.lower).double())==half_len
        upper_exist = torch.sum((X == self.upper).double())>0
        upper_cons = torch.sum((X<self.upper).double())==half_len
        bound = count_less==half_len
        bound_avg = self.lower+self.upper == bound*2

        median_in_cons = torch.logical_and(less_cons, more_cons)
        median_out_cons = torch.logical_and(torch.logical_and(bound, bound_avg), torch.logical_and(torch.logical_and(lower_cons, upper_cons), torch.logical_and(lower_exist, upper_exist)))
        return (median_in_cons*(count_equal!=0)+median_out_cons*(count_equal ==0), self.w)


    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

dummy output:  tensor(53.6250, dtype=torch.float64)


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_27165/3985421908.py:20: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if(len%2==0):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/utils.py:1686: UserWarning: The exported ONNX model failed ONNX shape inference. The model will not be executable by the ONNX Runtime. If this is unintended and you bel

In [7]:
# prover calculates settings, send to verifier

theory_output = torch.tensor(np.median(data))
lower_to_median = torch.tensor(np.sort(data)[int(len(data)/2)-1])
upper_to_median = torch.tensor(np.sort(data)[int(len(data)/2)])
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.lower = nn.Parameter(data = lower_to_median, requires_grad = False)
        self.upper = nn.Parameter(data = upper_to_median, requires_grad = False)
    def forward(self,X):
        # some expression of tolerance to error in the inference
        count_less = torch.sum((X < self.w).double())
        count_equal = torch.sum((X == self.w).double())
        len = X.size()[1]
        half_len = torch.div(len, 2)

        # doesnt support mod if not in conditional

        # For count_equal > 0
        if(len%2==0):
            is_odd = 0
        else:
            is_odd = 1
        less_cons = count_less<half_len+is_odd
        more_cons = count_less+count_equal>half_len

        # For count_equal == 0
        lower_exist = torch.sum((X == self.lower).double())>0
        lower_cons = torch.sum((X>self.lower).double())==half_len
        upper_exist = torch.sum((X == self.upper).double())>0
        upper_cons = torch.sum((X<self.upper).double())==half_len
        bound = count_less==half_len
        bound_avg = self.lower+self.upper == bound*2

        median_in_cons = torch.logical_and(less_cons, more_cons)
        median_out_cons = torch.logical_and(torch.logical_and(bound, bound_avg), torch.logical_and(torch.logical_and(lower_cons, upper_cons), torch.logical_and(lower_exist, upper_exist)))
        return (median_in_cons*(count_equal!=0)+median_out_cons*(count_equal ==0), self.w)
   
prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [0], "resources", settings_path)

Theory_output:  tensor(49., dtype=torch.float64)
==== Generate & Calibrate Setting ====
scale:  [0]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":0,"param_scale":0,"scale_rebase_multiplier":10,"lookup_range":[-78,196],"logrows":14,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":7540,"total_const_size":4,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,0],"model_input_scales":[0],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":[{"GreaterThan":{"a":0.0}},"KroneckerDelta"],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_27165/1250105436.py:24: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if(len%2==0):


In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0


Time setup: 1.362818956375122 seconds
Theory output:  tensor(49., dtype=torch.float64)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 49.0
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[10512373747352303962, 11798585516934984832, 13421675179368312123, 2200257403316998104], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [5937023402479976189, 6015681745618410760, 9712426403540212160, 757908509798626762]]], 'proof': '0bd41b8e851817ff5ba00dc1888985e1f66dd96dca691449eb9d7921b6d58067286e60fc1f0d1af51fc1802f171a370a71009195558abf3a90ee8b4bd9a5503a022ddc7564441cb227b21c3e51042bc1cc3f48713319b9e0c974ee6a6659b182155865449ada8287fa83a5545c90789b99ba9b09dfd7a802c046b1d8b68e62eb055c94a269bf9fcf9dbcb991db3f3de397ca7040b5ae30c1caab68078395292f1d4768ca5c42543bf0cdda319ac330007b628f76236c58291a760dea27892c94013cc01e9ac386ed26ac01fa16c9d36b1374a6836e79a8f963721aa6a17318951c544e64fd0b77074d2c591d458ed2029240b148ff02d12f3c8405e70b6a9a1a00f1deaf13bbd40dae76106bf15564c356c9d4076dddcc57a77444277b39b1aa18e04d586f3a3c8162f4bec4a8ede0040dc4434d4ec808c413ee560306f9b6df03094fd9c42946d5caf5c02a9183570128f477be95d058de35ac9d90f8aa3d6

In [9]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[10512373747352303962, 11798585516934984832, 13421675179368312123, 2200257403316998104], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [5937023402479976189, 6015681745618410760, 9712426403540212160, 757908509798626762]]]
proof boolean:  1.0
proof result 1 : 49.0
verified
